In [25]:
import numpy as np
from skimage.feature import graycomatrix, graycoprops

def calc_glcm_all_agls(img, label, props, dists=[5], agls=[0, np.pi/4, np.pi/2, 3*np.pi/4], lvl=256, sym=True, norm=False) -> list:
    """ 
    Calculate GLCM 

    Args:
        img (_type_): _description_
        label (_type_): _description_
        props (_type_): _description_
        dists (list, optional): _description_. Defaults to [5].
        agls (list, optional): _description_. Defaults to [0, np.pi/4, np.pi/2, 3*np.pi/4].
        lvl (int, optional): _description_. Defaults to 256.
        sym (bool, optional): _description_. Defaults to True.
        norm (bool, optional): _description_. Defaults to True.

    Returns:
        _type_: _description_
    """
    
    glcm = graycomatrix(img, 
                        distances=dists, 
                        angles=agls, 
                        levels=lvl,
                        symmetric=sym, 
                        normed=norm)
    feature = []
    glcm_props = [propery for name in props for propery in graycoprops(glcm, name)[0]]
    for item in glcm_props:
            feature.append(item)
    # feature.append(label) 
    
    return feature

In [26]:
IMG_PATH      = '../dataset/'    # Path to the dataset
IMG_DIMENTION = (1000, 1000)     # (width, height)
PROPERTY_GLCM = ['contrast', 'homogeneity', 'correlation', 'energy']

feature_extraction   = []

In [27]:
import os
import cv2

for idx, folder in enumerate(os.listdir(IMG_PATH)):
    files = os.listdir(os.path.join(IMG_PATH, folder))

    for file in files:
        if file.find(".jpg") > -1:
            image       = cv2.imread(os.path.abspath(os.path.join(IMG_PATH + folder, file)))
            image       = cv2.resize(image, IMG_DIMENTION, interpolation = cv2.INTER_AREA)
            
            # GLCM
            image_gray  = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
            feature_extraction.append(calc_glcm_all_agls(image_gray, folder, 
                                                         props=PROPERTY_GLCM, 
                                                         agls=[
                                                            #  0, 
                                                            #  np.pi/4, 
                                                            #  np.pi/2, 
                                                             3*np.pi/4
                                                        ]))

            # HSV
            image_hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
            h, s, v   = cv2.split(image_hsv)                    # Destruct value HSV
           
            # Calculate mean HSV
            h_mean = np.mean(h)
            s_mean = np.mean(s)
            v_mean = np.mean(v)

            # Append h_mean, s_mean, v_mean to feature_extraction
            feature_extraction[len(feature_extraction)-1].append(h_mean)
            feature_extraction[len(feature_extraction)-1].append(s_mean)
            feature_extraction[len(feature_extraction)-1].append(v_mean)

            # Append target, label
            feature_extraction[len(feature_extraction)-1].append(f'{idx}')
            feature_extraction[len(feature_extraction)-1].append(folder)



In [28]:
columns = []
# angles = ['0', '45', '90','135']
angles = ['135']
for name in PROPERTY_GLCM:
    for ang in angles:
        columns.append(name + "_" + ang)

columns.append("h_mean")
columns.append("s_mean")
columns.append("v_mean")
columns.append("target")
columns.append("label")

In [29]:
len(feature_extraction[0]), len(columns)

(9, 9)

In [30]:
import pandas as pd

df = pd.DataFrame(feature_extraction, columns=columns)
df.to_csv("result/feature_extraction_135_deg.csv", index=False)
df.head()

,contrast_135,homogeneity_135,correlation_135,energy_135,h_mean,s_mean,v_mean,target,label
0,173.188674,0.815761,0.802639,0.158778,102.019578,6.076716,239.601118,0,Skala0
1,146.395336,0.791353,0.935058,0.164909,107.860592,49.071800,219.021018,0,Skala0
2,137.333758,0.772817,0.914314,0.126759,107.707690,42.995309,214.469718,0,Skala0
3,176.692779,0.758480,0.889411,0.193896,111.507190,21.193654,218.849183,0,Skala0
4,120.837353,0.737244,0.860901,0.151010,108.038949,60.611979,215.771037,0,Skala0
